# JUICE SPICE Kernel Dataset Status
SKD version: {skd_version} <br>
Creation: {current_time} by ESA SPICE Service (ESAC/ESA). <br>


## Startup
Prepare the testing environment at first stage, so if any validation step fails stopping the execution, the tests are already defined and in fail state.

In [1]:
from spival.classes.history import TestHistory

test_history = TestHistory()
test_history.add_test('XM-C1', 'Metakernel is valid', 'Consistency')
test_history.add_test('XM-C2', 'Frame chain', 'Consistency')
test_history.add_test('XM-C3', 'Rotation matrices', 'Consistency')
test_history.add_test('XM-C4', 'Fields of view', 'Consistency')
test_history.add_test('XM-V5', 'OEM to SPK error', 'Validity/Error', threshold='~100m')
test_history.add_test('XM-Q4', 'JUICE Orientation (quaternions w.r.. J2000)', 'Quality/Data')
test_history.add_test('XM-Q5-MPO', 'JUICE Solar Array (SA) Angles', 'Quality/Data')
test_history.add_test('XM-Q1-MPO', 'JUICE SA Solar Aspect Angle', 'Quality/Data')
test_history.add_test('BC-Q2', 'JUICE Medium Gain Antenna Angles', 'Quality/Data')
test_history.add_test('BC-Q8', 'JUICE Medium Gain Antenna - Earth Angle', 'Quality/Data')

### XM-C1 - Test Metakernel is valid 
{XM-C1_description}

In [ ]:
from spiops import spiops
import spiceypy

spiops.load('{metakernel}')               
mission_config = spiops.load_config('{config_file}')               

test_history.set_test_result('XM-C1', True)

start_time = '{start_time_measured}'                                # Start time
finish_time ='{finish_time_measured}'                               # End time
resolution = {resolution}                                           # Resolution

interval = spiops.TimeWindow(start_time, finish_time, resolution=resolution) # spiops object TimeWindow generated
sun = spiops.Target('SUN', time=interval, frame='IAU_SUN')                   # spiops object Target Sun generated
juice = spiops.Observer('JUICE', time=interval, target=sun, mission_config=mission_config)  # spiops object Observer JUICE generated
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### <center><span style='color:Green'> Metakernel loaded successfully.  </span></center>

### XM-C2 - Frame chain
Ensure that frame changes operations can be performed at any time (incl. instrument frames), meaning all frames are connected.

In [ ]:
plan_mk = '{metakernel}'
spiops.load(plan_mk)
frm_start_time = '{start_time_measured}'                                # Start time
frm_finish_time ='{finish_time_measured}'                               # End time
num_samples = 100

all_frames_ok = spiops.check_frame_chain(frm_start_time, frm_finish_time, num_samples, ignore_frames=['@EW@_PLAN', '@EW@_MEAS'])
test_history.set_test_result('XM-C2', all_frames_ok)

### XM-C3 - Rotation matrices
Check that all TK frames defined with matrices are defined with proper rotation matrices.

In [ ]:
all_matrices_ok = spiops.check_rotation_matrices()
test_history.set_test_result('XM-C3', all_matrices_ok)


### XM-C4 - Fields of view
Check that all instruments fields of view are defined properly.

In [ ]:
all_fovs_ok = spiops.check_fovs(max_angle_deg=89.99994)
test_history.set_test_result('XM-C4', all_fovs_ok)
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

## Coverage

The coverage provided by the SPK and CK files is displayed for the Operational and the Planning meta-kernels. 

The Data type designation includes a reference to the originator of the data, the type of data and the reference period is provided. This is a three letter acronym. The first letter defines the file originator:
*    s: Science Operation
*    m: Mission Analysis
*    f: Flight Dynamic

the second letter defines the reference period of the data:
*    c: Cruise phase reference
*    l: Science phase Long term reference
*    m: Science phase Medium term reference
*    s: Science phase Short term reference
*    o: Undefined reference

the third letter indicates the type of data
*    <span style='color:Orange'> p: Predicted data</span>
*    <span style='color:Green'> r: Reconstructed data</span>
*    <span style='color:Red'> t: Test data</span>
*    <span style='color:Purple'> c: Commanded data (from Housekeeping Telemetry)</span>
*    <span style='color:Blue'> m: Measured data (from Housekeeping Telemetry)</span>


In [ ]:
spiops.ck_coverage_timeline('{metakernel}', ['JUICE_SPACECRAFT_PLAN', 'JUICE_SPACECRAFT_MEAS'])

In [ ]:
spiops.spk_coverage_timeline('{metakernel}', ['JUICE'])

### XM-V5 - OEM to SPK error
Comparison of spacecraft position and velocity between source OEM and generated SPK 

In [ ]:
spiops.load('{metakernel}')
target_spk = '{skd_path}/spk/{reconstructed_spk}'
max_pos_err, max_vel_err = spiops.spkVsOem('JUICE', target_spk, plot_style='line', notebook=True)
test_history.set_test_result('XM-V5', (max_pos_err != None) and (max_pos_err < 0.1))
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

## S/C Structures Orientation
Quaternions, JUICE Solar Array Orientation and Solar Aspect Angle, Medium Gain Antenna Orientation and MGA boresight-Earth Angle.

### XM-Q4 - JUICE Orientation (quaternions w.r.. J2000)

In [ ]:
spiops.load('{metakernel}')
juice.Plot('quaternions', notebook=True)
test_history.set_test_result('XM-Q4', True)

### XM-Q5-MPO - JUICE Solar Array (SA) Angles 

In [ ]:
juice.Plot('sa_ang', notebook=True)
test_history.set_test_result('XM-Q5-MPO', True)

### XM-Q1-MPO - JUICE SA Solar Aspect Angle

In [ ]:
juice.Plot('saa_sa', notebook=True)
test_history.set_test_result('XM-Q1-MPO', True)

### BC-Q2 - JUICE Medium Gain Antenna Angles

In [ ]:
juice.Plot('mga_angles', notebook=True)
test_history.set_test_result('BC-Q2', True)

### BC-Q8 - JUICE Medium Gain Antenna - Earth Angle

In [ ]:
juice.Plot('mga_earth', notebook=True)
test_history.set_test_result('BC-Q8', True)

In [ ]:
# Unload ops MK
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

In [ ]:
# Show validation results
test_history.show_tests()